# MMM Data Exploration and Analysis

This notebook demonstrates the exploration and analysis of real MMM data with mediation assumptions.


In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Import our custom modules
from data_preparation import DataPreparator
from utils import set_random_seed

# Set random seed for reproducibility
set_random_seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


## 1. Load Real MMM Data

We'll work with the actual 2-year weekly MMM dataset with the following characteristics:
- **Mediation Assumption**: Google spend mediates the relationship between social channels and revenue
- **Real Data**: Actual media spend, revenue, and business metrics
- **Time Period**: 2 years of weekly data (2023-09-17 to 2025-09-07)
- **Zero-spend Periods**: Real periods with no media spend
- **Business Variables**: Price, promotions, followers, email/SMS


In [ ]:
# Load the real MMM dataset
data = pd.read_csv('../data/raw/Assessment 2 - MMM Weekly.csv')

# Convert week column to datetime
data['date'] = pd.to_datetime(data['week'])
data = data.drop('week', axis=1)  # Remove the original week column

# Rename columns to match our expected format
data = data.rename(columns={
    'emails_send': 'email_volume',
    'sms_send': 'sms_volume',
    'social_followers': 'followers'
})

print(f"Loaded dataset shape: {data.shape}")
print(f"Date range: {data['date'].min()} to {data['date'].max()}")
print(f"Columns: {list(data.columns)}")
print(f"Total weeks: {len(data)}")


In [ ]:
# Display first few rows
print("First 10 rows of the dataset:")
data.head(10)


In [ ]:
# Basic statistics
print("Dataset Statistics:")
print("=" * 50)
data.describe()


## 2. Data Quality Assessment

Let's examine the data quality and identify any issues:


In [ ]:
# Check for missing values
missing_data = data.isnull().sum()
print("Missing values by column:")
print(missing_data[missing_data > 0])

if missing_data.sum() == 0:
    print("\n✅ No missing values found!")
else:
    print(f"\n⚠️ Total missing values: {missing_data.sum()}")


In [ ]:
# Check for zero-spend periods
media_cols = [col for col in data.columns if 'spend' in col.lower()]
print("Zero-spend periods by channel:")
for col in media_cols:
    zero_count = (data[col] == 0).sum()
    zero_pct = (zero_count / len(data)) * 100
    print(f"{col}: {zero_count} periods ({zero_pct:.1f}%)")


## 3. Time Series Analysis

Let's examine the time series characteristics of our key variables:


In [ ]:
# Plot time series of key variables
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

# Revenue
axes[0].plot(data['date'], data['revenue'], color='blue', linewidth=2)
axes[0].set_title('Revenue Over Time')
axes[0].set_ylabel('Revenue ($)')
axes[0].tick_params(axis='x', rotation=45)

# Google Spend
axes[1].plot(data['date'], data['google_spend'], color='red', linewidth=2)
axes[1].set_title('Google Spend Over Time')
axes[1].set_ylabel('Google Spend ($)')
axes[1].tick_params(axis='x', rotation=45)

# Social Media Spend
axes[2].plot(data['date'], data['facebook_spend'], label='Facebook', alpha=0.8)
axes[2].plot(data['date'], data['tiktok_spend'], label='TikTok', alpha=0.8)
axes[2].plot(data['date'], data['instagram_spend'], label='Instagram', alpha=0.8)
axes[2].plot(data['date'], data['snapchat_spend'], label='Snapchat', alpha=0.8)
axes[2].set_title('Social Media Spend Over Time')
axes[2].set_ylabel('Spend ($)')
axes[2].legend()
axes[2].tick_params(axis='x', rotation=45)

# Average Price
axes[3].plot(data['date'], data['average_price'], color='green', linewidth=2)
axes[3].set_title('Average Price Over Time')
axes[3].set_ylabel('Price ($)')
axes[3].tick_params(axis='x', rotation=45)

# Promotions
axes[4].plot(data['date'], data['promotions'], color='orange', linewidth=2)
axes[4].set_title('Promotions Over Time')
axes[4].set_ylabel('Promotion (0/1)')
axes[4].tick_params(axis='x', rotation=45)

# Followers
axes[5].plot(data['date'], data['followers'], color='purple', linewidth=2)
axes[5].set_title('Followers Over Time')
axes[5].set_ylabel('Followers')
axes[5].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 4. Correlation Analysis

Let's examine the relationships between variables:


In [ ]:
# Calculate correlation matrix
correlation_matrix = data.select_dtypes(include=[np.number]).corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of MMM Variables')
plt.tight_layout()
plt.show()


In [ ]:
# Focus on revenue correlations
revenue_correlations = correlation_matrix['revenue'].sort_values(ascending=False)
print("Correlations with Revenue:")
print(revenue_correlations)


## 5. Mediation Analysis Preview

Let's examine the mediation assumption - how social channels influence Google spend:


In [ ]:
# Scatter plots showing social channels vs Google spend
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

social_channels = ['facebook_spend', 'tiktok_spend', 'instagram_spend', 'snapchat_spend']
colors = ['blue', 'red', 'green', 'orange']

for i, (channel, color) in enumerate(zip(social_channels, colors)):
    row = i // 2
    col = i % 2
    
    axes[row, col].scatter(data[channel], data['google_spend'], alpha=0.6, color=color)
    axes[row, col].set_xlabel(f'{channel.replace("_", " ").title()}')
    axes[row, col].set_ylabel('Google Spend')
    
    # Add trend line
    z = np.polyfit(data[channel], data['google_spend'], 1)
    p = np.poly1d(z)
    axes[row, col].plot(data[channel], p(data[channel]), "r--", alpha=0.8)
    
    # Calculate correlation
    corr = data[channel].corr(data['google_spend'])
    axes[row, col].set_title(f'{channel.replace("_", " ").title()} vs Google\\nCorrelation: {corr:.3f}')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate mediation strength
print("Mediation Analysis - Social Channels to Google Spend:")
print("=" * 50)

for channel in social_channels:
    # Simple correlation as proxy for mediation strength
    corr = data[channel].corr(data['google_spend'])
    print(f"{channel.replace('_', ' ').title()}: {corr:.3f}")

print("\nGoogle Spend to Revenue:")
google_revenue_corr = data['google_spend'].corr(data['revenue'])
print(f"Google Spend: {google_revenue_corr:.3f}")


## 6. Save Data

Let's save the processed data for use in subsequent notebooks:


In [ ]:
# Save the processed data
data.to_csv('../data/raw/mmm_data.csv', index=False)
print("✅ Data saved to ../data/raw/mmm_data.csv")

# Display summary statistics
print("\n📊 Data Summary:")
print(f"- Total weeks: {len(data)}")
print(f"- Date range: {data['date'].min().strftime('%Y-%m-%d')} to {data['date'].max().strftime('%Y-%m-%d')}")
print(f"- Average weekly revenue: ${data['revenue'].mean():,.0f}")
print(f"- Total media spend: ${data[media_cols].sum().sum():,.0f}")
print(f"- Average price: ${data['average_price'].mean():.2f}")
print(f"- Promotion periods: {data['promotions'].sum()} weeks ({data['promotions'].mean()*100:.1f}%)")
print(f"- Total followers: {data['followers'].max():,.0f}")
print(f"- Email volume: {data['email_volume'].mean():,.0f} per week")
print(f"- SMS volume: {data['sms_volume'].mean():,.0f} per week")
